# Creating a covariate dataset that encodes categorical sample variables

The sample data possesses covariates as shown in <a href='https://github.com/cognoma/cancer-data/blob/master/3.explore-mutations.ipynb'>this notebook</a>. These may provide a spurious signal that a classifier accommodates for, and could confound attempts to pick out the actual signal that we desire. This notebook will create a file with encoded information on these covariates. Classifiers being implemented by the machine learning group can use this as additional data to train on.

In [1]:
import os

import pandas as pd
import numpy as np

Let's peak at the sample data.

In [2]:
path = os.path.join('data', 'samples.tsv')
covariates_df = pd.read_table(path, index_col=0)
covariates_df = covariates_df.rename(columns={'organ_of_origin': 'organ'})
covariates_df.head(10)

,patient_id,sample_type,disease,acronym,organ,gender,age_diagnosed,dead,days_survived,recurred,days_recurrence_free,n_mutations
sample_id,,,,,,,,,,,,
TCGA-02-0047-01,TCGA-02-0047,Primary Tumor,glioblastoma multiforme,GBM,Brain,Male,78.0,1.0,448.0,NaN,NaN,39
TCGA-02-0055-01,TCGA-02-0055,Primary Tumor,glioblastoma multiforme,GBM,Brain,Female,62.0,1.0,76.0,NaN,NaN,33
TCGA-02-2483-01,TCGA-02-2483,Primary Tumor,glioblastoma multiforme,GBM,Brain,Male,43.0,0.0,466.0,NaN,NaN,24
TCGA-02-2485-01,TCGA-02-2485,Primary Tumor,glioblastoma multiforme,GBM,Brain,Male,53.0,0.0,470.0,NaN,NaN,39
TCGA-02-2486-01,TCGA-02-2486,Primary Tumor,glioblastoma multiforme,GBM,Brain,Male,64.0,0.0,493.0,NaN,NaN,39
TCGA-04-1348-01,TCGA-04-1348,Primary Tumor,ovarian serous cystadenocarcinoma,OV,Ovary,Female,44.0,1.0,1483.0,NaN,NaN,39
TCGA-04-1357-01,TCGA-04-1357,Primary Tumor,ovarian serous cystadenocarcinoma,OV,Ovary,Female,52.0,NaN,NaN,NaN,NaN,32
TCGA-04-1362-01,TCGA-04-1362,Primary Tumor,ovarian serous cystadenocarcinoma,OV,Ovary,Female,59.0,1.0,1348.0,NaN,NaN,57
TCGA-05-4244-01,TCGA-05-4244,Primary Tumor,lung adenocarcinoma,LUAD,Lung,Male,70.0,NaN,NaN,NaN,NaN,196


Let's get a sense of how much missing data there is.

In [3]:
print('Total number of samples: {:,}'.format(len(covariates_df)))
print('Number of nulls in each column:')
covariates_df.isnull().sum(axis=0)

Total number of samples: 7,306
Number of nulls in each column:


patient_id                 0
sample_type                0
disease                    0
acronym                    0
organ                      0
gender                    19
age_diagnosed             50
dead                     111
days_survived            111
recurred                1638
days_recurrence_free    1638
n_mutations                0
dtype: int64

In [4]:
# Specify which variables are categorical
categorical_variables = ['acronym', 'organ', 'gender', 'dead', 'recurred']

# Number of categories per categorical variable
covariates_df[categorical_variables].apply(lambda x: x.nunique())

acronym     32
organ       28
gender       2
dead         2
recurred     2
dtype: int64

In [5]:
covariates_df.drop(['patient_id', 'sample_type', 'disease'], axis=1, inplace=True)
print('Remaining variables to encode: %s' % ', '.join(categorical_variables))

Remaining variables to encode: acronym, organ, gender, dead, recurred


Before encoding, we're going to use the disease categories below. So let's store them for later use.

Inspecting the head of the covariates DataFrame above, we see that two columns, namely <code>dead</code> and <code>recurred</code>, need some attention. We're going to encode categorical variables using panda's get_dummies. Since these columns are indicated by a 1 or 0, this will become the column header when encoded, as below.

In [6]:
pd.get_dummies(covariates_df, columns=categorical_variables).head(2)

,age_diagnosed,days_survived,days_recurrence_free,n_mutations,acronym_ACC,acronym_BLCA,acronym_BRCA,acronym_CESC,acronym_CHOL,acronym_COAD,...,organ_Testis,organ_Thymus,organ_Thyroid Gland,organ_Uterus,gender_Female,gender_Male,dead_0.0,dead_1.0,recurred_0.0,recurred_1.0
sample_id,,,,,,,,,,,,,,,,,,,,,
TCGA-02-0047-01,78.0,448.0,NaN,39,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
TCGA-02-0055-01,62.0,76.0,NaN,33,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


Let's rename the values in each of these so that they more accurately reflect the underlying variable.

In [7]:
renamer = {
    'dead_0.0': 'alive',
    'dead_1.0': 'dead',
    'recurred_0.0': 'has_not_recurred',
    'recurred_1.0': 'has_recurred',
    'gender_Female': 'female',
    'gender_Male': 'male',
}
covariates_df = pd.get_dummies(covariates_df, columns=categorical_variables).rename(columns=renamer)
covariates_df.head(2)

,age_diagnosed,days_survived,days_recurrence_free,n_mutations,acronym_ACC,acronym_BLCA,acronym_BRCA,acronym_CESC,acronym_CHOL,acronym_COAD,...,organ_Testis,organ_Thymus,organ_Thyroid Gland,organ_Uterus,female,male,alive,dead,has_not_recurred,has_recurred
sample_id,,,,,,,,,,,,,,,,,,,,,
TCGA-02-0047-01,78.0,448.0,NaN,39,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
TCGA-02-0055-01,62.0,76.0,NaN,33,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


Now the column name more accurately reflects the underlying variable. The categorical values are encoded as numeric data that can be input to the types of classifiers that we have been using.

Another useful covariate will be the logarithm plus one function of the number mutations that was calculated in the aforementioned notebook.

In [8]:
covariates_df['n_mutations_log1p'] = np.log1p(covariates_df.pop('n_mutations'))
covariates_df.head(2)

,age_diagnosed,days_survived,days_recurrence_free,acronym_ACC,acronym_BLCA,acronym_BRCA,acronym_CESC,acronym_CHOL,acronym_COAD,acronym_DLBC,...,organ_Thymus,organ_Thyroid Gland,organ_Uterus,female,male,alive,dead,has_not_recurred,has_recurred,n_mutations_log1p
sample_id,,,,,,,,,,,,,,,,,,,,,
TCGA-02-0047-01,78.0,448.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,3.688879
TCGA-02-0055-01,62.0,76.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,3.526361


Finally, let's save this to a <code>.tsv</code> file.

In [9]:
path = os.path.join('data', 'covariates.tsv')
covariates_df.to_csv(path, sep='\t', float_format='%.5g')